# Lab 7 Exercises - Logistic Regression, K-NN & Naive Bayes Classifier



The US adult income dataset was extracted by Barry Becker from the 1994 US Census Database. The data set consists of anonymous information such as occupation, age, native country, race, capital gain, capital loss, education, work class and more. Each row is labelled as either having a salary greater than ">50K" or "<=50K".

The goal here is to train the logistic regression, K-NN & Naive Bayes classifiers on the training dataset to predict the column `income_bracket` which has two possible values ">50K" and "<=50K" and compare the accuracy of each classifier with the test dataset.

Note that the dataset is made up of categorical and continuous features. It also contains missing values
The categorical columns are: `workclass, education, marital_status, occupation, relationship, race, gender, native_country`

The continuous columns are: `age, education_num, capital_gain, capital_loss, hours_per_week`

**Dataset columns**

```
age: continuous.

workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.

fnlwgt: continuous.

education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.

education-num: continuous.

marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.

occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.

relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.

race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.

sex: Female, Male.

capital-gain: continuous.

capital-loss: continuous.

hours-per-week: continuous.

native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

income-bracket: <=50k, >50k
```

*Note: for the purposes of this exercise, fnlwgt (final weight, a value assigned by the census bureau as part of their sampling methodology of census data across the 51 states) can be dropped for simplicity*

**In this exercise we are going to use `%timeit` magic command to compare the runtime of the different algorithms together. We are going to use it once while building the model and once while scoring the model** 

The `%timeit` magic command is used measure the execution time of a piece of code. In order to use it, write the command just before the relevant of code, in the same line to measure the execution time.

It returns the mean and standard deviation of code run time calculated over `r` number of runs and `n` number of loops within each run (*it may return different results for each time you run the cell*).

<br/>
<br/>

Timeit Function: [Official Python Documentation - timeit function](https://docs.python.org/3/library/timeit.html)

%timeit Magic Command - Ipython: [ipython - timeit magic command](https://ipython.readthedocs.io/en/stable/interactive/magics.html)

Extra Links on %timeit Magic Command: [nkme - timeit magic command](https://note.nkmk.me/en/python-timeit-measure)

**Example**

In [1]:
%timeit [num for num in range(20)]

1.2 µs ± 117 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In the above example, the list comprehension was evaluated for 7 runs with each run having 1 million loops (default behaviour). This took an average of 2.05 microseconds with a standard deviation of 31.8 nanoseconds (*it may return different results for each time you run the cell*).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use("seaborn")

In [3]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

## Dataset Loading & Cleaning

**Dataset Loading**

Make sure the datasets have been loaded correctly. You may want to have a look at the [pd.read_csv documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) to resolve dataset loading issues.

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/GUC-DM/W2022/main/data/census_income.csv')
df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


**Check the unique values for each categorical attribute**

In [5]:
categorical_col = []
for column in df.columns: 
   if df[column].dtype == object: 
     categorical_col.append (column)
     print(f"{column} : {df[column].unique()}")
     print("======================================")

workclass : ['State-gov' 'Self-emp-not-inc' 'Private' 'Federal-gov' 'Local-gov' '?'
 'Self-emp-inc' 'Without-pay' 'Never-worked']
education : ['Bachelors' 'HS-grad' '11th' 'Masters' '9th' 'Some-college' 'Assoc-acdm'
 'Assoc-voc' '7th-8th' 'Doctorate' 'Prof-school' '5th-6th' '10th'
 '1st-4th' 'Preschool' '12th']
marital_status : ['Never-married' 'Married-civ-spouse' 'Divorced' 'Married-spouse-absent'
 'Separated' 'Married-AF-spouse' 'Widowed']
occupation : ['Adm-clerical' 'Exec-managerial' 'Handlers-cleaners' 'Prof-specialty'
 'Other-service' 'Sales' 'Craft-repair' 'Transport-moving'
 'Farming-fishing' 'Machine-op-inspct' 'Tech-support' '?'
 'Protective-serv' 'Armed-Forces' 'Priv-house-serv']
relationship : ['Not-in-family' 'Husband' 'Wife' 'Own-child' 'Unmarried' 'Other-relative']
race : ['White' 'Black' 'Asian-Pac-Islander' 'Amer-Indian-Eskimo' 'Other']
sex : ['Male' 'Female']
native_country : ['United-States' 'Cuba' 'Jamaica' 'India' '?' 'Mexico' 'South'
 'Puerto-Rico' 'Honduras' 'En

**Handle inconsistencies and missing values**

In [6]:
categorical_col = []
for column in df.columns: 
   if df[column].dtype == object: 
     categorical_col.append (column)
     print(f"{column} : {df[column].unique()}")
     print("======================================")

workclass : ['State-gov' 'Self-emp-not-inc' 'Private' 'Federal-gov' 'Local-gov' '?'
 'Self-emp-inc' 'Without-pay' 'Never-worked']
education : ['Bachelors' 'HS-grad' '11th' 'Masters' '9th' 'Some-college' 'Assoc-acdm'
 'Assoc-voc' '7th-8th' 'Doctorate' 'Prof-school' '5th-6th' '10th'
 '1st-4th' 'Preschool' '12th']
marital_status : ['Never-married' 'Married-civ-spouse' 'Divorced' 'Married-spouse-absent'
 'Separated' 'Married-AF-spouse' 'Widowed']
occupation : ['Adm-clerical' 'Exec-managerial' 'Handlers-cleaners' 'Prof-specialty'
 'Other-service' 'Sales' 'Craft-repair' 'Transport-moving'
 'Farming-fishing' 'Machine-op-inspct' 'Tech-support' '?'
 'Protective-serv' 'Armed-Forces' 'Priv-house-serv']
relationship : ['Not-in-family' 'Husband' 'Wife' 'Own-child' 'Unmarried' 'Other-relative']
race : ['White' 'Black' 'Asian-Pac-Islander' 'Amer-Indian-Eskimo' 'Other']
sex : ['Male' 'Female']
native_country : ['United-States' 'Cuba' 'Jamaica' 'India' '?' 'Mexico' 'South'
 'Puerto-Rico' 'Honduras' 'En

In [7]:
df['income_bracket'] = df['income_bracket'].str.replace('<=50K.', "<=50K")
df['income_bracket'] = df['income_bracket'].str.replace('>50K.', ">50K")

In [8]:
df['native_country'].mode()[0]

'United-States'

In [9]:
df['occupation'].mode()[0]

'Prof-specialty'

In [10]:
df['workclass'].mode()[0]

'Private'

In [11]:
df['native_country'] = df['native_country'].str.replace('?', "United-States")
df['occupation'] = df['occupation'].str.replace('?', "Prof-speciality")
df['workclass'] = df['workclass'].str.replace('?', "Private")

In [12]:
categorical_col = []
for column in df.columns: 
   if df[column].dtype == object: 
     categorical_col.append (column)
     print(f"{column} : {df[column].unique()}")
     print("======================================")

workclass : ['State-gov' 'Self-emp-not-inc' 'Private' 'Federal-gov' 'Local-gov'
 'Self-emp-inc' 'Without-pay' 'Never-worked']
education : ['Bachelors' 'HS-grad' '11th' 'Masters' '9th' 'Some-college' 'Assoc-acdm'
 'Assoc-voc' '7th-8th' 'Doctorate' 'Prof-school' '5th-6th' '10th'
 '1st-4th' 'Preschool' '12th']
marital_status : ['Never-married' 'Married-civ-spouse' 'Divorced' 'Married-spouse-absent'
 'Separated' 'Married-AF-spouse' 'Widowed']
occupation : ['Adm-clerical' 'Exec-managerial' 'Handlers-cleaners' 'Prof-specialty'
 'Other-service' 'Sales' 'Craft-repair' 'Transport-moving'
 'Farming-fishing' 'Machine-op-inspct' 'Tech-support' 'Prof-speciality'
 'Protective-serv' 'Armed-Forces' 'Priv-house-serv']
relationship : ['Not-in-family' 'Husband' 'Wife' 'Own-child' 'Unmarried' 'Other-relative']
race : ['White' 'Black' 'Asian-Pac-Islander' 'Amer-Indian-Eskimo' 'Other']
sex : ['Male' 'Female']
native_country : ['United-States' 'Cuba' 'Jamaica' 'India' 'Mexico' 'South' 'Puerto-Rico'
 'Hondura

In [13]:
df['income_bracket'] = df['income_bracket'].astype('category').cat.codes

For the sake of simplicty, we won't be using the fnlwgt attribute

**Drop the `fnlwgt` attribute from the dataset**

In [14]:
df.drop('fnlwgt', axis = 'columns', inplace = True)

## Exploratory Data Analysis

**Q: Does education have an impact on a person's income bracket?**

*Hint: see lab 6 for an example of plotting a categorical attribute against a binary attribute.*

In [15]:
df['income_bracket'] = df['income_bracket'].str.replace('>50K.', '50K.', regex = True)
df.income_bracket.unique()

AttributeError: ignored

## Data Preprocessing

**Encode the categorical columns using an appropriate strategy**

In [16]:
df_encode = pd.get_dummies(df, columns = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country'])
df_encode

,age,education_num,capital_gain,capital_loss,hours_per_week,income_bracket,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,...,native_country_Portugal,native_country_Puerto-Rico,native_country_Scotland,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia
0,39,13,2174,0,40,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,13,0,0,13,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,9,0,0,40,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,53,7,0,0,40,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,28,13,0,0,40,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,13,0,0,36,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
48838,64,9,0,0,40,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
48839,38,13,0,0,50,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
48840,44,13,5455,0,40,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


**Apply Normalization on the numerical Columns**

In [17]:
numerical_columns = ['age', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']
df_encode[numerical_columns] = (
    df_encode[numerical_columns] - df_encode[numerical_columns].min()    
) / (
    df_encode[numerical_columns].max() - df_encode[numerical_columns].min()
)
df_encode.head()

,age,education_num,capital_gain,capital_loss,hours_per_week,income_bracket,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,...,native_country_Portugal,native_country_Puerto-Rico,native_country_Scotland,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia
0,0.301370,0.800000,0.02174,0.0,0.397959,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0.452055,0.800000,0.00000,0.0,0.122449,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0.287671,0.533333,0.00000,0.0,0.397959,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,0.493151,0.400000,0.00000,0.0,0.397959,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,0.150685,0.800000,0.00000,0.0,0.397959,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## Modelling

**Split the data into training and testing sets, and then train the Logistic Regression, K-NN, and Naive Bayes Classifiers on the training set** (Use %timeit when building the model)

[Scikit-learn API Reference](https://scikit-learn.org/stable/modules/classes.html)

**Use the `timeit` magic command.**

The `timeit` magic command is used to measure the execution time for the small python code snippets. This command runs the code a million times (by default) to get the most precise value for the code execution time​.

It returns the mean and standard deviation of code run time calculated over `r` number of runs and `n` number of loops within each run (*it may return different results for each time you run the cell*).

**Please note that, for this dataset you need to set the `max_iter` attribute of logistic regression to at least `2000` so that the model will be able converge, meaning that it could find a local optimal solution.**

Logistic regression fails to converge with this dataset if we run it for too few iterations. It could also be that the dataset is not linearly separable. Since this is a linear model, validation techinques like the ones used in lab 5 can be used.The logistic regression model is an iterative algorithm that is fitted in multiple iterations. It might not converge properly if the input variables are not normalized properly. In this case, it needs more iterations to be able to converge and find a solution.

So, you can set `max_iter` attribute of logistic regression to a larger value. The default is `1000`. This should be your last resort. So, if the algorithm does not converge within the first 1000 iterations, you need to set the `max_iter` to a larger value.


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X = df_encode.drop('income_bracket',axis=1)
y = df_encode['income_bracket']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

logreg = LogisticRegression()
logreg.fit(X_train, y_train);

In [19]:
y_pred = logreg.predict(X_test)
print('Accuracy of Logistic Regression on Test Set: {:.2f}'.format(logreg.score(X_test , y_test)))

Accuracy of Logistic Regression on Test Set: 0.85


In [20]:
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=7)

In [21]:
knn.predict(X_test)
print('Accuracy of KNN model:', knn.score(X_test, y_test))

Accuracy of KNN model: 0.8339645818405159


In [22]:
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)

In [23]:
from sklearn import metrics 
print("Gaussian Naive Bayes model accuracy:" , metrics.accuracy_score(y_test, y_pred)*100)

Gaussian Naive Bayes model accuracy: 61.75657692701403


## Evaluation

**Evaluate each model using their built-in `score` function to get them evaluated based on the `accuracy` metric**

**Use `%timeit` when scoring the model**

In [24]:
%timeit logreg.fit(X_train, y_train)

1.57 s ± 258 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%timeit gnb.fit(X_train , y_train)

72.7 ms ± 561 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
%timeit knn.fit(X_train , y_train)

12.7 ms ± 95 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## References

Dataset source: https://archive.ics.uci.edu/ml/datasets/census+income